# Meta explainer network for Tox21 dataset  

## step 0: whats the scope?

* Binary task: **SR-MMP active (1) vs inactive (0)**
* Input: SMILES  +  Tier-1 RDKit phys-chem descriptors (30 dims)
* Model: ChemBERTa CLS (frozen)  ⊕  descriptors  →  2-layer head
* SHAP: attribute across CLS & desc block
* Meta-MLP: 10 CLS dims  + 5 top descriptors = **15-dim** input

## Step 1: Dataset Curation 

In [2]:
import pandas as pd

raw = pd.read_csv("Data_v2/original/tox21.csv")
target = "SR-MMP"
df = raw[~raw[target].isna()].copy()
df[target] = df[target].astype(int)
df.reset_index(drop=True, inplace=True)
print("Dataset size:", df.shape)


Dataset size: (5810, 14)


## Step 2: Generate Molecular Fingerprints using the SMILES 

To match our ADE pipeline architecture, we’ll compute ECFP4 (Extended-Connectivity Fingerprints) for each SMILES string.

This is equivalent to the input features you used in the previous app — except we used SHAP values instead of raw bits.

In [3]:
from rdkit.Chem import AllChem
from rdkit import DataStructs

def smiles_to_ecfp(smiles, radius=2, n_bits=2048):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# Generate features
X_list, y_list, bad_ids = [], [], []
for i, row in df.iterrows():
    fp = smiles_to_ecfp(row["smiles"])
    if fp is not None:
        X_list.append(fp)
        y_list.append(row[target])
    else:
        bad_ids.append(i)

print(f"✅ Processed: {len(X_list)} | ❌ Failed: {len(bad_ids)}")



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Coding Projects\Predicting-Drug-Response-Us

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Coding Projects\Predicting-Drug-Response-Us

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Coding Projects\Predicting-Drug-Response-Us

...


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Coding Projects\Predicting-Drug-Response-Us

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\libs\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Coding Projects\Predicting-Drug-Response-Us

AttributeError: _ARRAY_API not found

NameError: name 'Chem' is not defined